**Load Libraries**

In [68]:
import os
import pandas as pd
import numpy as np
from tensorflow.keras import layers, models, Input
from tensorflow.keras.optimizers import Adam

**Load Data**

In [69]:
database = pd.read_csv("BANKACCOUNTDATA.csv")

**GAN**
1) Generator (Nueral Net that creates what it thinks is replicated data)
2) Discriminator (Nueral Net that is fed both the real and fake data and chooses which one is the most realistis)
3) Real/Fake (Sends Back Propogration to the Nueral Nets to edit them depending on the outcome of the Discriminator)

**Helper Functions**

In [70]:
# Generate Noise for Generator
def latent_dim():
    #Generate function to optimize randomness
    return 10

#alpha function for LeakyReLU paramater
def alpha():
    #Generate function to optimize dying nuerons
    return 0.2

#input shape function for discriminator
def inputShape(dataFrame):
    return len(dataFrame.columns)-1

#drop paramater for discriminator
def dropout():
    return 0.3

#Define batch size
def batch():
    return 32

#Data values
def meanIncome(dataFrame):
    return dataFrame["DEPOSIT"].mean()

def stddevIncome(dataFrame):
    return dataFrame["DEPOSIT"].std()

def meanExpense(dataFrame):
    return dataFrame["WITHDRAWAL"].mean()

def stddevExpense(dataFrame):
    return dataFrame["WITHDRAWAL"].std()

**Generator**

In [71]:
# Generator Function
def generator(noise):
    model = models.Sequential()
    model.add(layers.Dense(128, activation='relu', input_shape=(noise,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=alpha()))
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=alpha()))
    model.add(layers.Dense(8, activation='tanh'))
    return model


**Discriminator**

In [72]:
# Discriminator Function
def discriminator(inputShape):
    model = models.Sequential()
    model.add(layers.Dense(256, activation='relu', input_shape=(inputShape,)))
    model.add(layers.LeakyReLU(alpha=alpha()))
    model.add(layers.Dropout(dropout()))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.LeakyReLU(alpha=alpha()))
    model.add(layers.Dropout(dropout()))
    model.add(layers.Dense(1, activation='sigmoid'))
    return model
    

**Defining GAN Model**

In [73]:
# Initialize models
generator_model = generator(latent_dim())
discriminator_model = discriminator(inputShape(database))

# Compile the discriminator
discriminator_model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Freeze the discriminator weights before compiling the GAN
discriminator_model.trainable = False

# Define the GAN input
gan_input = Input(shape=(latent_dim(),))

# Pass the input through the generator to create fake data
generated_data = generator_model(gan_input)

# Pass the generated data through the frozen discriminator
gan_output = discriminator_model(generated_data)

# Create the GAN model with the generator and discriminator
gan = models.Model(gan_input, gan_output)

# Compile the GAN with binary crossentropy loss
gan.compile(optimizer=Adam(), loss='binary_crossentropy')

# Display model summaries to verify the structure
generator_model.summary()
discriminator_model.summary()
gan.summary()

c:\Users\Tyler Jackson\01 - SCHOOL\Programs\Anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\Tyler Jackson\01 - SCHOOL\Programs\Anaconda3\Lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Model: "sequential_22"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_60 (Dense)                │ (None, 128)            │         1,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_22          │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_38 (LeakyReLU)      │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_61 (Dense)                │ (None, 256)            │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_23          │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_39 (LeakyReLU)      │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_62 (Dense)                │ (None, 8)              │         2,056 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 38,024 (148.53 KB)

 Trainable params: 37,256 (145.53 KB)

...

Model: "functional_152"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_32 (InputLayer)     │ (None, 10)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_22 (Sequential)      │ (None, 8)              │        38,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_23 (Sequential)      │ (None, 1)              │        35,329 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 73,353 (286.54 KB)

 Trainable params: 37,256 (145.53 KB)

 Non-trainable params: 36,097 (141.00 KB)

**Generate Real Data**

In [74]:
# Function to generate real data for training
def generateRealData(batch):
    income = np.random.normal(loc=meanIncome(database), scale=stddevIncome(database), size=batch)
    expense = np.random.normal(loc=meanExpense(database), scale=stddevExpense(database), size=batch)
    return np.stack((income, expense), axis=1)

**Train GAN**

In [75]:
# Function to train the GAN
def train_gan(generator, discriminator, gan, latent_dim, epochs, batch_size):
    # Training loop
    for epoch in range(epochs):
        # Generate 'Real Data'
        real_data = generateRealData(batch_size)
        real_labels = np.ones((batch_size, 1))

        # Generate Fake Data
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        fake_data = generator.predict(noise)
        fake_labels = np.zeros((batch_size, 1))

        # Train discriminator
        discriminator_loss_real = discriminator.train_on_batch(real_data, real_labels)
        discriminator_loss_fake = discriminator.train_on_batch(fake_data, fake_labels)
        discriminator_loss = 0.5 * np.add(discriminator_loss_real, discriminator_loss_fake)

        # Train generator
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        misleading_labels = np.ones((batch_size, 1))  # Labels for generator training
        generator_loss = gan.train_on_batch(noise, misleading_labels)

        # Print progress
        if epoch % 10 == 0:  # Adjust frequency of output
            print(f"Epoch {epoch}, Discriminator Loss: {discriminator_loss}, Generator Loss: {generator_loss}")

# Run training
train_gan(generator_model, discriminator_model, gan, latent_dim(), 100, batch())

KeyError: 'DEPOSIT'